## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---
## First, I'll compute the camera calibration using chessboard images

In [1]:
import pickle
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import time


In [2]:
class Lane_detect():
    # init method or constructor   
    def __init__(self): 
        self.dist_pickle_=[]
        self.mtx_=[]
        self.dist_=[]
        self.frame_=[]
        self.orignal_frame_=[]
        self.h_=int
        self.w_=int
        self.c_=int
        
        self.M=[]
        self.Minv_=[]

        self.left_fit_=[]
        self.right_fit_=[]
        self.prev_left_=[]
        self.prev_right_=[]


    """  cam_clibration is used to load the calibration matrix mtx,dist """  
    def cam_clibration(self,cam_clibration_dir):
        self.dist_pickle_ = pickle.load( open( cam_clibration_dir, "rb" ) )
        self.mtx_ = self.dist_pickle_["mtx"]
        self.dist_ = self.dist_pickle_["dist"]

        
    """  feed_image is used to load the image and set the undistort version """  
    def feed_image(self,frame):
         self.h_, self.w_, self.c_ = frame.shape
         self.frame_ = cv2.undistort(frame, self.mtx_, self.dist_, None, self.mtx_)
         self.orignal_frame_=self.frame_

         
    """  yellow_thresh is used to get the yellow lane in the Image"""  
    def yellow_thresh(self,frame, show=False):
        
        mask = np.zeros((self.h_, self.w_), np.uint8)
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HLS)
        low_yellow = np.array([0, 120, 90])
        high_yellow = np.array([35, 200, 255])
        ymask = cv2.inRange(hsv, low_yellow, high_yellow)
        output = cv2.bitwise_or(mask, ymask, mask=ymask)
        if show:
            cv2.imshow('yellow', output)
            if cv2.waitKey(1) == ord('q'):
                cv2.destroyAllWindows()
        return output

    """  white_thresh is used to get the white lane in the Image"""  
    def white_thresh(self,frame, show=False):
        
        mask = np.zeros((self.h_, self.w_), np.uint8)
        hls = cv2.cvtColor(frame, cv2.COLOR_BGR2HLS)
        low_white = np.array([0, 180, 10])
        high_white = np.array([255, 255, 255])
        ymask = cv2.inRange(hls, low_white, high_white)
        output = cv2.bitwise_or(mask, ymask, mask=ymask)
        if show:
            cv2.imshow('white', output)
            if cv2.waitKey(1) == ord('q'):
                cv2.destroyAllWindows()
        return output

    """  binary is used to get the binary of lane based on color threshold(white,yellow) in the Image"""
    def binary(self, show=False):
        if show:
            output1 = self.white_thresh(self.frame_, True)
            output2 = self.yellow_thresh(self.frame_, True)      
        else:
            output1 = self.white_thresh(self.frame_, False)
            output2 = self.yellow_thresh(self.frame_, False)

        self.frame_ = cv2.bitwise_or(output2, output1)
        if show:
            cv2.imshow('output', self.frame_)
            if cv2.waitKey(30) == ord('q'):
                cv2.destroyAllWindows()
    """  perspective_view is used to transform the Image view from normal to Bird eye view"""    
    def perspective_view(self, show=False):
        
        src = np.float32(
            [[200, self.h_-1],
            [565, 450],
                [722, 450],
                [1100, self.h_-1]
            ])
        dst = np.float32(
            [[100, self.h_-1],
            [100, 0],
                [1000, 0],
                [1000, self.h_-1]
            ])

        self.M_ = cv2.getPerspectiveTransform(src, dst)
        self.Minv_ = cv2.getPerspectiveTransform(dst, src)

        self.frame_ = cv2.warpPerspective(self.frame_, self.M_, (self.w_, self.h_), flags=cv2.INTER_LINEAR)
        if show:
            cv2.imshow("perspective_view", self.frame_)
            if cv2.waitKey(1) == ord('q'):
                cv2.destroyAllWindows()

    """  find_lane is used to find the lane points based on a sliding window and fit a polyfit from the second degree""" 
    def find_lane(self):
        nonzero = self.frame_.nonzero()
        nonzero_y = np.array(nonzero[0])
        nonzero_x = np.array(nonzero[1])
        margin = 100
        num_windows = 9
        histogram = np.sum(self.frame_[int(self.h_/2):, :], axis=0)
        midpoint = int(self.h_/2)
        win_height = int(self.h_/num_windows)
        left_x_base = np.argmax(histogram[:midpoint])
        right_x_base = np.argmax(histogram[midpoint:])+midpoint
        current_left_x = left_x_base
        current_right_x = right_x_base
        minpix = 10
        left_point_ind = []
        right_point_ind = []
        for window in range(num_windows):
            win_y_low = self.h_-(window+1)*win_height
            win_y_high = win_y_low+win_height
            x_left_low = current_left_x-margin
            x_left_high = current_left_x+margin
            x_right_low = current_right_x-margin
            x_right_high = current_right_x+margin
            good_left_inds = ((win_y_low < nonzero_y) & (win_y_high > nonzero_y) & (
                x_left_low < nonzero_x) & (x_left_high > nonzero_x)).nonzero()[0]
            good_right_inds = ((nonzero_y >= win_y_low) & (nonzero_y < win_y_high) & (
                nonzero_x >= x_right_low) & (nonzero_x < x_right_high)).nonzero()[0]
            left_point_ind.append(good_left_inds)
            right_point_ind.append(good_right_inds)
            if len(good_left_inds) > minpix:
                current_left_x = np.int(np.mean(nonzero_x[good_left_inds]))
            if len(good_right_inds) > minpix:
                current_right_x = np.int(np.mean(nonzero_x[good_right_inds]))
        left_point_ind = np.concatenate(left_point_ind)
        right_point_ind = np.concatenate(right_point_ind)
        leftx = nonzero_x[left_point_ind]
        lefty = nonzero_y[left_point_ind]
        rightx = nonzero_x[right_point_ind]
        righty = nonzero_y[right_point_ind]
        
        if (lefty.size != 0) & (leftx.size != 0):
            self.left_fit_ = np.polyfit(lefty, leftx, 2)
        else:
            self.left_fit_ = self.prev_left_

        if (righty.size != 0) & (rightx.size != 0):
            self.right_fit_ = np.polyfit(righty, rightx, 2)
        else:
            self.right_fit_ = self.prev_right_

        self.prev_right_ = self.right_fit_
        self.prev_left_ = self.left_fit_

    """  draw is used to draw the lane line on the feeded Image and get the car position""" 
    def draw(self,show,color=(0,255,0)):      
        ploty = np.linspace(0, self.h_-1, self.h_)
        left_fitx = self.left_fit_[0]*ploty**2 + self.left_fit_[1]*ploty + self.left_fit_[2]
        right_fitx = self.right_fit_[0]*ploty**2 + self.right_fit_[1]*ploty + self.right_fit_[2]

        pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
        pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
        pts = np.hstack((pts_left, pts_right))
        x = np.int_([pts])
        cv2.fillPoly(self.frame_, np.int_([pts]), 255)

        warpedbinary = cv2.warpPerspective(self.frame_, self.Minv_, (self.w_, self.h_))

        nonzero = warpedbinary.nonzero()
        nonzero_y = np.array(nonzero[0])
        nonzero_x = np.array(nonzero[1])

        y_high = max(nonzero_y)
        x_min = np.min(np.where(warpedbinary[y_high, :] == 255))
        x_max = np.max(np.where(warpedbinary[y_high, :] == 255))
        shift = ((x_min+x_max)/2)-(self.w_/2)

        warp_zero = np.zeros_like(warpedbinary).astype(np.uint8)
        color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
        cv2.fillPoly(color_warp, x, color)
        newwarp = cv2.warpPerspective(color_warp,  self.Minv_, (self.w_, self.h_))
        result = cv2.addWeighted(self.orignal_frame_, 1, newwarp, 0.3, 0)
        if shift > 0:
            direction = 'left'
        elif shift < 0:
            direction = 'right'
        else:
            direction = 'center'
        font = cv2.FONT_HERSHEY_SIMPLEX
        result = cv2.putText(result, 'Vehicle is {:.2f}pixels '.format(abs(
            shift))+direction+' of center', (10, 150), font, 2, (255, 255, 255), 2, cv2.LINE_AA)
        if show:
            cv2.imshow("result",result)
            if cv2.waitKey(1)==ord('q'):
                cv2.destroyAllWindows()
        return result
        

In [4]:
cap=cv2.VideoCapture("../project_video.mp4")
lane_detect=Lane_detect()
lane_detect.cam_clibration("cam_clibration")
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

# Define the codec and create VideoWriter object.The output is stored in 'outpy.avi' file.
out = cv2.VideoWriter('Lande_detection_result.mp4',cv2.VideoWriter_fourcc('M','J','P','G'), 30, (frame_width,frame_height))
while (cap.isOpened()):
    stime = time.time()
    ret, frame = cap.read()
    
    if ret:
        cv2.imshow("frame",frame)
        if cv2.waitKey(1)==ord('q'):
                break
        lane_detect.feed_image(frame)
        lane_detect.binary(False)
        lane_detect.perspective_view(False)
        lane_detect.find_lane()
        result=lane_detect.draw(False)
        out.write(result)
        
    else:
        break  
    
cv2.destroyAllWindows() 

ValueError: zero-size array to reduction operation minimum which has no identity